In [15]:
# Minibatch Training file

In [9]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [45]:
import torch.nn.functional as F
from fastai import datasets
import gzip, pickle, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from torch.nn import init
from torch import nn
import torch

In [46]:
mpl.rcParams['image.cmap'] = 'gray'

In [47]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [48]:
x_train, y_train, x_valid, y_valid = get_data()

In [49]:
def normalize(x, m, s): return (x-m)/s

In [50]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [51]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
    
    def __call__(self,x):
        for l in self.layers: x = l(x)
        return x

In [52]:
# m = 784
# n = 50000 train 
# c = tensor(10)

model = Model(m,nh,10)

In [53]:
pred = model(x_train)

In [54]:
pred

tensor([[-0.1962,  0.0249,  0.0640,  ..., -0.0660, -0.0347,  0.0037],
        [-0.2577,  0.0173, -0.0375,  ..., -0.0763,  0.0021,  0.1523],
        [-0.2657,  0.0343, -0.0174,  ..., -0.0267, -0.1410,  0.1721],
        ...,
        [-0.2314,  0.0630,  0.0093,  ..., -0.1249, -0.1041,  0.0310],
        [-0.0778,  0.1314, -0.0108,  ..., -0.0532, -0.0503, -0.0159],
        [-0.2298, -0.0126,  0.0611,  ..., -0.0770, -0.0477,  0.0772]],
       grad_fn=<AddmmBackward>)

# cross entropy loss



In [55]:
def log_softmax(x): return (x.exp()/(x.exp().sum(-1, keepdim=True))).log()

In [56]:
sm_pred = log_softmax(pred)

In [57]:
sm_pred

tensor([[-2.5106, -2.2895, -2.2503,  ..., -2.3804, -2.3491, -2.3107],
        [-2.5812, -2.3063, -2.3610,  ..., -2.3998, -2.3215, -2.1713],
        [-2.5859, -2.2860, -2.3376,  ..., -2.3470, -2.4613, -2.1482],
        ...,
        [-2.5234, -2.2290, -2.2827,  ..., -2.4169, -2.3961, -2.2610],
        [-2.3859, -2.1767, -2.3189,  ..., -2.3613, -2.3583, -2.3240],
        [-2.5414, -2.3242, -2.2504,  ..., -2.3885, -2.3593, -2.2343]],
       grad_fn=<LogBackward>)

In [58]:
y_train[:3] 

tensor([5, 0, 4])

* We want to find out what's the probabily associated with y_train and sm_pred

In [59]:
sm_pred.shape

torch.Size([50000, 10])

In [60]:
sm_pred[0] # In this go to 5th i.e. -2.1836 that's the answer we gonna wanna want

tensor([-2.5106, -2.2895, -2.2503, -2.2843, -2.3759, -2.1311, -2.1941, -2.3804,
        -2.3491, -2.3107], grad_fn=<SelectBackward>)

In [61]:
# what we want
sm_pred[0][5]

tensor(-2.1311, grad_fn=<SelectBackward>)

In [62]:
# Integer array indexing
sm_pred[[0,1,2], [5,0,4]] # pass two list....first is row and second list column

tensor([-2.1311, -2.5812, -2.1891], grad_fn=<IndexBackward>)

In [63]:
y_train.shape[0], range(y_train.shape[0])

(50000, range(0, 50000))

In [64]:
# range(target.shape[0]) is basically all row ...50k
def nll(input, target): return -input[range(target.shape[0]), target].mean()

In [65]:
loss = nll(sm_pred, y_train)

In [66]:
loss

tensor(2.3115, grad_fn=<NegBackward>)

In [67]:
def log_softmax(x): return x - x.exp().sum(-1, keepim=True).log()

In [68]:
# numeric stability trick
def logsumexp(x):
    m = x.max(-1)[0]
    return m + (x-m[:,None]).exp().sum(-1).log()

In [69]:
# it's such an important method so it's in pytorch

def log_softmax(x): return x - x.logsumexp(-1, keepdim=True)

# nll(log(softmax(pred), y_train), loss)  --> This is cross entropy
# F.log_softmax
# F.nll_loss -> Pytorch
F.cross_entropy??
# test_near(F.cross_entropy(pred, y_train),loss)

Signature:
F.cross_entropy(
    input,
    target,
    weight=None,
    size_average=None,
    ignore_index=-100,
    reduce=None,
    reduction='mean',
)
Source:   
def cross_entropy(input, target, weight=None, size_average=None, ignore_index=-100,
                  reduce=None, reduction='mean'):
    # type: (Tensor, Tensor, Optional[Tensor], Optional[bool], int, Optional[bool], str) -> Tensor
    r"""This criterion combines `log_softmax` and `nll_loss` in a single
    function.

    See :class:`~torch.nn.CrossEntropyLoss` for details.

    Args:
        input (Tensor) : :math:`(N, C)` where `C = number of classes` or :math:`(N, C, H, W)`
            in case of 2D Loss, or :math:`(N, C, d_1, d_2, ..., d_K)` where :math:`K \geq 1`
            in the case of K-dimensional loss.
        target (Tensor) : :math:`(N)` where each value is :math:`0 \leq \text{targets}[i] \leq C-1`,
            or :math:`(N, d_1, d_2, ..., d_K)` where :math:`K \geq 1` for
            K-dimensional loss.
    

# Basic Training Loop

In [70]:
loss_func = F.cross_entropy

In [71]:
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()
# can't take mean of ints in pytorch so float conversion

In [72]:
bs = 64 #batch size

xb = x_train[0:bs]   # mini-batch from x
preds = model(xb)
preds[0], preds.shape

(tensor([-0.1962,  0.0249,  0.0640,  0.0301, -0.0615,  0.1833,  0.1202, -0.0660,
         -0.0347,  0.0037], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [73]:
yb = y_train[0:bs]
loss_func(preds,yb)

tensor(2.3044, grad_fn=<NllLossBackward>)

In [75]:
accuracy(preds,yb)  # Our accuracy is 0.09 1% because we haven't trained our model yet

tensor(0.0938)

In [76]:
# so we have model basically that is giving random answer...So we need to train it

In [77]:
# now go to lesson SGD /nbs/dl1/lesson2-sgd.ipynb to explain abi

In [87]:
a = (50000-1)//64
a + 1

782

In [119]:
lr = 0.5
epochs = 1

In [120]:
for epoch in range(epochs):
    for i in range((n-1)//bs+1):
        # n-1 don't understand  782 -- should be 784
        # set_traces
        start_i = i*bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss = loss_func(model(xb), yb)
        
        loss.backward()
        with torch.no_grad():
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias -= l.bias.grad * lr
                    l.weight.grad.zero_()
                    l.bias  .grad.zero_()

In [121]:
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.0014, grad_fn=<NllLossBackward>), tensor(1.))

In [122]:
# Above is too much code so make it small

In [123]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in, nh)
        self.l2 = nn.Linear(nh, n_out)
        
    def __call__(self,x): return self.l2(F.relu(self.l1(x)))

In [124]:
model = Model(m, nh, 10)

In [125]:
model

Model(
  (l1): Linear(in_features=784, out_features=50, bias=True)
  (l2): Linear(in_features=50, out_features=10, bias=True)
)